# NeRF Training on Google Colab

**GitHub Repo**: https://github.com/mabuhann/nerf-pytorch

**Expected Runtime**: 4-6 hours on T4 GPU

## Step 1: Check GPU

**Make sure you enabled GPU runtime!**

Runtime → Change runtime type → GPU

In [ ]:
!nvidia-smi

import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("\n⚠️ WARNING: No GPU detected! Go to Runtime → Change runtime type → GPU")

## Step 2: Clone GitHub Repo

In [ ]:
import os

# Clean up any existing directory
!rm -rf nerf-pytorch

# Clone the repo
!git clone https://github.com/mabuhann/nerf-pytorch.git

# Change to the directory
%cd /content/nerf-pytorch

print("\n✓ Repo cloned")
print(f"Working directory: {os.getcwd()}")
print("\nFiles:")
!ls -lh

## Step 3: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("\n✓ Drive mounted")

## Step 4: Copy Dataset from Drive

In [ ]:
# Create data directory structure
!mkdir -p data/nerf_synthetic

# Copy dataset from your Drive
# Your dataset is at: /content/drive/MyDrive/data/nerf_synthetic/lego
!cp -r /content/drive/MyDrive/data/nerf_synthetic/lego data/nerf_synthetic/

print("\n✓ Dataset copied")

# Verify the dataset
print("\nDataset contents:")
!ls -lh data/nerf_synthetic/lego/

## Step 5: Install Dependencies

In [ ]:
!pip install imageio imageio-ffmpeg configargparse scikit-image -q

print("✓ Dependencies installed")

## Step 6: Verify Everything is Ready

In [ ]:
import json
import os

print("Verification check:\n")
print(f"Current directory: {os.getcwd()}")

# Check Python files
print("\nPython files:")
required = ['run_nerf_with_metrics.py', 'run_nerf_helpers.py', 'load_blender.py']
all_good = True
for f in required:
    exists = os.path.exists(f)
    print(f"{'✓' if exists else '✗'} {f}")
    if not exists:
        all_good = False

# Check config
print("\nConfig file:")
config_exists = os.path.exists('configs/lego_config.txt')
print(f"{'✓' if config_exists else '✗'} configs/lego_config.txt")
if not config_exists:
    all_good = False

# Check dataset
print("\nDataset:")
for split in ['train', 'val', 'test']:
    path = f'data/nerf_synthetic/lego/transforms_{split}.json'
    exists = os.path.exists(path)
    if exists:
        with open(path) as f:
            data = json.load(f)
        print(f"✓ {split}: {len(data['frames'])} images")
    else:
        print(f"✗ {split}: not found at {path}")
        all_good = False

# Check actual image files
print("\nImage files:")
for split in ['train', 'val', 'test']:
    img_dir = f'data/nerf_synthetic/lego/{split}'
    if os.path.exists(img_dir):
        n_images = len([f for f in os.listdir(img_dir) if f.endswith('.png')])
        print(f"✓ {split}: {n_images} .png files")
    else:
        print(f"✗ {split} directory not found")
        all_good = False

if all_good:
    print("\n" + "="*60)
    print("✓ ALL CHECKS PASSED - READY TO TRAIN!")
    print("="*60)
else:
    print("\n" + "="*60)
    print("✗ SOME CHECKS FAILED - FIX ERRORS BEFORE TRAINING")
    print("="*60)

## Step 7: Start Training

**⚠️ This will take 4-6 hours!**

**Tips:**
- Keep this browser tab open
- Or use Colab Pro for longer sessions
- Training will print progress every 100 iterations
- Checkpoints saved every 10,000 iterations
- Test evaluation every 25,000 iterations

In [ ]:
# Start training
!python run_nerf_with_metrics.py --config configs/lego_config.txt

## Step 8: View Results

In [ ]:
import json
from IPython.display import Image, display

# Load and display summary
print("="*60)
print("TRAINING SUMMARY")
print("="*60)

with open('logs/lego_metrics/summary_metrics.json', 'r') as f:
    summary = json.load(f)

for key, value in summary.items():
    if value is not None:
        print(f"{key}: {value}")

print("="*60)

# Display training plot
print("\nTraining Metrics:")
if os.path.exists('logs/lego_metrics/training_metrics.png'):
    display(Image('logs/lego_metrics/training_metrics.png'))
else:
    print("Plot not found")

## Step 9: Detailed Analysis (Optional)

In [ ]:
# Run detailed analysis if available
if os.path.exists('analyze_results.py'):
    !python analyze_results.py \
        --log_dir logs/lego_metrics \
        --save_plot logs/lego_metrics/detailed_analysis.png
    
    # Display detailed plot
    print("\nDetailed Analysis:")
    if os.path.exists('logs/lego_metrics/detailed_analysis.png'):
        display(Image('logs/lego_metrics/detailed_analysis.png'))
else:
    print("analyze_results.py not found - skipping detailed analysis")

## Step 10: Download Results

In [ ]:
from google.colab import files

# Zip all results
print("Zipping results...")
!zip -r results.zip logs/lego_metrics/

print("\nStarting download...")
files.download('results.zip')

print("\n✓ Download complete!")
print("\nThe zip file contains:")
print("  - summary_metrics.json (key metrics)")
print("  - training_metrics.npz (all training data)")
print("  - test_metrics.npz (test evaluations)")
print("  - training_metrics.png (plots)")
print("  - Model checkpoints (.tar files)")

## Step 11: Key Metrics for Your Report

In [ ]:
# Extract key metrics for report
with open('logs/lego_metrics/summary_metrics.json', 'r') as f:
    metrics = json.load(f)

print("="*60)
print("KEY METRICS FOR YOUR REPORT")
print("="*60)
print(f"\nTraining Time: {metrics['total_training_time_hours']:.2f} hours")
print(f"Test PSNR: {metrics['best_test_psnr']:.2f} dB")
print(f"Test SSIM: {metrics['avg_test_ssim']:.4f}")
print(f"Average Step Time: {metrics['avg_step_time']:.4f} seconds")
print("\n" + "="*60)
print("\nCopy these values to compare with 3DGS!")
print("="*60)

---

## Troubleshooting

### Out of Memory?
Run this to reduce memory usage, then restart from Step 7:

```python
# Reduce batch sizes
with open('configs/lego_config.txt', 'r') as f:
    config = f.read()

config = config.replace('N_rand = 1024', 'N_rand = 512')
config = config.replace('chunk = 8192', 'chunk = 4096')

with open('configs/lego_config.txt', 'w') as f:
    f.write(config)

print("✓ Config updated for lower memory")
```

### Session Timeout?
- Use Colab Pro for 24-hour sessions
- Keep browser tab active
- Checkpoints are saved every 10k iterations - you can resume if needed

### Files Not Found?
- Make sure you ran all cells in order
- Check Step 6 verification passed all checks
- Verify your Drive path is correct in Step 4